In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math
import random

# ---- CONFIG ----
N = 10000  # number of timesteps (1 Hz sampling ~ 2.8 hours)
ts0 = datetime.now()
dt = 1.0  # seconds

rng = np.random.default_rng(42)

# Blade types (from user's PDF list + common forms)
blade_types = [
    "Delta Form", "S Form", "4-Cut Form",
    "Krämer & Grebe 233", "Krämer & Grebe 423",
    "Laska M4S", "Seydelmann-BW"
]

# ---- Helper functions ----
def smooth_noise(n, scale=1.0, seed=None):
    r = np.cumsum(rng.normal(0, scale, n))
    r = (r - r.min()) / (r.max() - r.min() + 1e-9)  # 0..1
    r = (r - 0.5) * 2  # -1..1
    return r

def clamp(x, lo, hi):
    return np.minimum(np.maximum(x, lo), hi)

# ---- Timeline & base arrays ----
t = np.array([ts0 + timedelta(seconds=i) for i in range(N)])
sec = np.arange(N)

# Operating cycles: alternate load/no-load windows
cycle_len = 300  # 5 minutes per cycle
in_load = (sec % cycle_len) > (cycle_len * 0.2)  # 80% load, 20% idle

# Blade lifecycle schedule (state machine over time)
# Simulate more realistic wear progression with some randomness
wear_rate = 0.8 + 0.4 * smooth_noise(N, 0.1)  # Varying wear rate
p = np.cumsum(wear_rate) / np.max(np.cumsum(wear_rate))  # Normalized wear

blade_state = np.where(p < 0.6, "Sharp",
                np.where(p < 0.8, "Minor Wear",
                np.where(p < 0.95, "Major Wear", "Crack Detected")))

# Select a blade type for this run (or rotate types per cycle)
bt_idx = (sec // cycle_len) % len(blade_types)
blade_type_series = np.array([blade_types[i] for i in bt_idx])

# Blade type characteristics (each type has different baseline properties)
blade_properties = {
    "Delta Form": {"vib_base": 0.5, "torque_base": 20, "wear_resistance": 1.2},
    "S Form": {"vib_base": 0.6, "torque_base": 22, "wear_resistance": 1.0},
    "4-Cut Form": {"vib_base": 0.55, "torque_base": 21, "wear_resistance": 1.1},
    "Krämer & Grebe 233": {"vib_base": 0.65, "torque_base": 24, "wear_resistance": 0.9},
    "Krämer & Grebe 423": {"vib_base": 0.7, "torque_base": 25, "wear_resistance": 0.8},
    "Laska M4S": {"vib_base": 0.6, "torque_base": 23, "wear_resistance": 1.0},
    "Seydelmann-BW": {"vib_base": 0.55, "torque_base": 22, "wear_resistance": 1.3}
}

# Apply blade type properties
blade_vib_mu = np.zeros(N)
blade_torque_mu = np.zeros(N)
wear_resistance = np.zeros(N)

for i, blade_type in enumerate(blade_type_series):
    props = blade_properties[blade_type]
    blade_vib_mu[i] = props["vib_base"]
    blade_torque_mu[i] = props["torque_base"]
    wear_resistance[i] = props["wear_resistance"]

# Adjust baselines by condition
condition_multiplier = np.select(
    [blade_state=="Sharp", blade_state=="Minor Wear", blade_state=="Major Wear", blade_state=="Crack Detected"],
    [1.0, 1.5, 2.2, 3.5]
)

blade_vib_mu = blade_vib_mu * condition_multiplier
blade_vib = blade_vib_mu + rng.normal(0, 0.05, N) + 0.05 * smooth_noise(N, 0.3)

blade_torque_mu = blade_torque_mu * condition_multiplier
blade_torque = blade_torque_mu + (in_load.astype(float) * rng.normal(0.0, 1.2, N)) + (~in_load).astype(float)*rng.normal(-3.0, 1.0, N)
blade_torque = clamp(blade_torque, 5, 60)

blade_speed_mu = np.select(
    [blade_state=="Sharp", blade_state=="Minor Wear", blade_state=="Major Wear", blade_state=="Crack Detected"],
    [1200, 1180, 1160, 1100]
)
blade_speed = blade_speed_mu + rng.normal(0, 6, N) - (in_load.astype(float) * rng.normal(4, 2, N))

blade_noise_mu = np.select(
    [blade_state=="Sharp", blade_state=="Minor Wear", blade_state=="Major Wear", blade_state=="Crack Detected"],
    [68, 74, 78, 85]
)
blade_noise = blade_noise_mu + rng.normal(0, 1.0, N) + 0.5 * smooth_noise(N, 0.4)

blade_temp = np.select(
    [blade_state=="Sharp", blade_state=="Minor Wear", blade_state=="Major Wear", blade_state=="Crack Detected"],
    [35, 38, 42, 48]
).astype(float)
blade_temp += (in_load.astype(float) * 1.5) + rng.normal(0, 0.6, N)
blade_temp = clamp(blade_temp, 25, 90)

# RUL (cycles remaining) - more realistic calculation
# Base RUL depends on blade type and condition
base_rul = np.select(
    [blade_state=="Sharp", blade_state=="Minor Wear", blade_state=="Major Wear", blade_state=="Crack Detected"],
    [500, 300, 100, 20]
).astype(float)

# Apply blade type wear resistance
base_rul = base_rul * wear_resistance

# Stress factors that accelerate wear
vib_stress = clamp((blade_vib - 0.5) / 2.0, 0, 1)
torque_stress = clamp((blade_torque - 20) / 20.0, 0, 1)
temp_stress = clamp((blade_temp - 35) / 30.0, 0, 1)

# Combined stress factor
stress_factor = 0.4 * vib_stress + 0.4 * torque_stress + 0.2 * temp_stress

# RUL decreases based on stress and time
rul_decrease_rate = 1.0 + 2.0 * stress_factor
remaining_fraction = 1 - p
blade_rul = np.maximum(1, base_rul * remaining_fraction / rul_decrease_rate)

# ---- Motor signals (coupled to blade) ----
# Motor speed near 1500 (50Hz, 2-pole ~ 3000 synchronous; assuming gearbox; keep 1500 base with small slip)
motor_speed = 1500 - 0.02*(blade_torque-20) + rng.normal(0, 5, N)

# Motor vibration: base + coupling to blade vibration + bearing events
bearing_event = (np.sin(2*np.pi*sec/2000)+rng.normal(0,0.2,N) > 0.95)  # sparse
motor_vib = 0.45 + 0.35*(blade_vib-0.6) + 0.5*bearing_event.astype(float) + rng.normal(0, 0.05, N)
motor_vib = clamp(motor_vib, 0.2, 4.0)

# 3-phase currents: proportional to torque + small unbalance + electrical fault events
elec_fault_event = (np.sin(2*np.pi*sec/2500+1.0)+rng.normal(0,0.25,N) > 1.1)  # sparse
I_base = 10 + 0.25*(blade_torque-20)  # load coupling
unbal = rng.normal(0, 0.5, (N,3))
I_A = I_base + unbal[:,0] + 2.0*elec_fault_event.astype(float)
I_B = I_base + unbal[:,1] - 2.0*elec_fault_event.astype(float)
I_C = I_base + unbal[:,2] + rng.normal(0, 0.2, N)
I_A = clamp(I_A, 5, 40); I_B = clamp(I_B, 5, 40); I_C = clamp(I_C, 5, 40)

# Motor temperature: base + load + ambient + overheating events (caused by sustained high torque & vib)
overheat_index = (blade_torque > 30).astype(float) + (motor_vib > 1.2).astype(float)
overheat = (overheat_index > 1).astype(float)
motor_temp = 44 + 0.35*(I_base-10) + 2.0*overheat + rng.normal(0, 0.8, N)
motor_temp = clamp(motor_temp, 30, 110)

# Health state derivation from signals (rules)
health = np.full(N, "Normal", dtype=object)
health[(motor_temp > 60) & (I_base > 12)] = "Overheating"
health[(elec_fault_event)] = "Electrical Fault"
health[(motor_vib > 1.1) & (overheat == 0)] = "Bearing Fault"
health[(blade_torque > 32) & (motor_vib > 1.2)] = "Load Imbalance"

# Motor RUL from health indicators
health_penalty = np.where(health=="Normal", 0.0,
                   np.where(health=="Bearing Fault", 0.3,
                   np.where(health=="Electrical Fault", 0.7,
                   np.where(health=="Overheating", 1.2, 1.5))))

# Base motor RUL decreases with time but at different rates based on health
motor_rul_base = 1000 * (1 - p)  # Base RUL decreases linearly with time
motor_rul = np.maximum(1, motor_rul_base / (1 + health_penalty + 0.3*clamp((motor_vib-0.5),0,2) + 0.2*clamp((motor_temp-45)/30,0,2)))

# Add power consumption and efficiency metrics
power_factor = 0.85 + 0.1 * smooth_noise(N, 0.1)  # Varying power factor
power_consumption = (np.sqrt(3) * (I_A+I_B+I_C)/3 * 400 * power_factor / 1000).round(2)  # kW

# ---- Build DataFrames ----
motor_df = pd.DataFrame({
    "Timestamp": t,
    "PhaseA_Current": I_A.round(3),
    "PhaseB_Current": I_B.round(3),
    "PhaseC_Current": I_C.round(3),
    "Power_Consumption": power_consumption,
    "Power_Factor": power_factor.round(3),
    "Vibration": motor_vib.round(3),
    "Temp": motor_temp.round(3),
    "Speed": motor_speed.round(1),
    "Health_Status": health,
    "RUL": motor_rul.round(1)
})

blade_df = pd.DataFrame({
    "Timestamp": t,
    "Blade_Type": blade_type_series,
    "Vibration": blade_vib.round(3),
    "Torque": blade_torque.round(3),
    "Speed": blade_speed.round(1),
    "Noise": blade_noise.round(2),
    "Temp": blade_temp.round(2),
    "Condition": blade_state,
    "RUL": blade_rul.round(1)
})

combined_df = pd.DataFrame({
    "Timestamp": t,
    "Motor_Current": (0.333*(I_A+I_B+I_C)).round(3),
    "Motor_Vibration": motor_vib.round(3),
    "Motor_Temp": motor_temp.round(3),
    "Motor_Power": power_consumption,
    "Blade_Type": blade_type_series,
    "Blade_Vibration": blade_vib.round(3),
    "Blade_Torque": blade_torque.round(3),
    "Blade_Speed": blade_speed.round(1),
    "Blade_Noise": blade_noise.round(2),
    "Blade_Temp": blade_temp.round(2),
    "Health_Status": health,
    "Condition": blade_state,
    "RUL_Motor": motor_rul.round(1),
    "RUL_Blade": blade_rul.round(1)
})

# ---- Save ----
motor_path = "motor_dataset_realistic.csv"
blade_path = "blade_dataset_realistic.csv"
combined_path = "combined_dataset_realistic.csv"

motor_df.to_csv(motor_path, index=False)
blade_df.to_csv(blade_path, index=False)
combined_df.to_csv(combined_path, index=False)

motor_path, blade_path, combined_path

('motor_dataset_realistic.csv',
 'blade_dataset_realistic.csv',
 'combined_dataset_realistic.csv')

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# ---- CONFIG ----
N = 10000  # timesteps
ts0 = datetime.now()
rng = np.random.default_rng(42)

# Blade types
blade_types = [
    "Delta Form", "S Form", "4-Cut Form",
    "Krämer & Grebe 233", "Krämer & Grebe 423",
    "Laska M4S", "Seydelmann-BW"
]

# Condition → RUL ranges
blade_rul_ranges = {
    "Sharp": (400, 600),
    "Minor Wear": (150, 300),
    "Major Wear": (50, 120),
    "Crack Detected": (1, 30)
}
motor_rul_ranges = {
    "Normal": (700, 1000),
    "Bearing Fault": (300, 500),
    "Electrical Fault": (100, 250),
    "Overheating": (20, 80),
    "Load Imbalance": (20, 80)
}

# ---- Helpers ----
def clamp(x, lo, hi):
    return np.minimum(np.maximum(x, lo), hi)

def smooth_noise(n, scale=1.0):
    r = np.cumsum(rng.normal(0, scale, n))
    r = (r - r.min()) / (r.max() - r.min() + 1e-9)  # normalize
    return (r - 0.5) * 2  # -1..1

def gen_piecewise_rul(states, ranges):
    n = len(states)
    rul = np.zeros(n)

    # find transition points where state changes
    idx = [0] + [i for i in range(1, n) if states[i] != states[i-1]] + [n]
    idx = sorted(set(idx))  # ensure increasing order

    current_rul = random.uniform(*ranges[states[0]])

    for i in range(len(idx) - 1):
        seg_len = idx[i+1] - idx[i]
        if seg_len <= 0:
            continue

        lo, hi = ranges[states[idx[i+1]-1]]
        end_rul = random.uniform(lo, hi)

        seg_rul = np.linspace(current_rul, end_rul, seg_len)
        rul[idx[i]:idx[i+1]] = seg_rul
        current_rul = end_rul

    return np.maximum.accumulate(rul[::-1])[::-1]  # force monotonic


# ---- Timeline ----
t = np.array([ts0 + timedelta(seconds=i) for i in range(N)])
sec = np.arange(N)

# Operating cycles
cycle_len = 300
in_load = (sec % cycle_len) > (cycle_len * 0.2)

# Wear progression → blade condition
p = np.linspace(0, 1, N)
blade_state = np.where(p < 0.6, "Sharp",
                np.where(p < 0.8, "Minor Wear",
                np.where(p < 0.95, "Major Wear", "Crack Detected")))

# Blade types rotate per cycle
bt_idx = (sec // cycle_len) % len(blade_types)
blade_type_series = np.array([blade_types[i] for i in bt_idx])

# ---- Blade Signals ----
blade_vib = np.select(
    [blade_state=="Sharp", blade_state=="Minor Wear", blade_state=="Major Wear", blade_state=="Crack Detected"],
    [0.5, 1.0, 1.8, 3.0]
).astype(float) + rng.normal(0, 0.05, N)

blade_torque = np.select(
    [blade_state=="Sharp", blade_state=="Minor Wear", blade_state=="Major Wear", blade_state=="Crack Detected"],
    [20, 24, 28, 34]
).astype(float) + in_load.astype(float) * rng.normal(0, 1.0, N)
blade_torque = clamp(blade_torque, 10, 60)

blade_speed = np.select(
    [blade_state=="Sharp", blade_state=="Minor Wear", blade_state=="Major Wear", blade_state=="Crack Detected"],
    [1200, 1180, 1160, 1100]
).astype(float) + rng.normal(0, 4, N)

blade_noise = np.select(
    [blade_state=="Sharp", blade_state=="Minor Wear", blade_state=="Major Wear", blade_state=="Crack Detected"],
    [68, 74, 78, 85]
).astype(float) + rng.normal(0, 1.0, N)

blade_temp = np.select(
    [blade_state=="Sharp", blade_state=="Minor Wear", blade_state=="Major Wear", blade_state=="Crack Detected"],
    [35, 38, 42, 48]
).astype(float) + in_load.astype(float) * 1.5 + rng.normal(0, 0.5, N)
blade_temp = clamp(blade_temp, 25, 90)

# ---- Motor Signals ----
motor_speed = 1500 - 0.02*(blade_torque-20) + rng.normal(0, 3, N)

motor_vib = 0.4 + 0.4*(blade_vib-0.5) + rng.normal(0, 0.05, N)
motor_vib = clamp(motor_vib, 0.2, 4.0)

I_base = 10 + 0.25*(blade_torque-20)
I_A = I_base + rng.normal(0, 0.5, N)
I_B = I_base + rng.normal(0, 0.5, N)
I_C = I_base + rng.normal(0, 0.5, N)
I_A, I_B, I_C = clamp(I_A, 5, 40), clamp(I_B, 5, 40), clamp(I_C, 5, 40)

motor_temp = 44 + 0.35*(I_base-10) + rng.normal(0, 0.7, N)
motor_temp = clamp(motor_temp, 30, 110)

# Motor health rules
health = np.full(N, "Normal", dtype=object)
health[(motor_temp > 70)] = "Overheating"
health[(motor_vib > 1.3) & (motor_temp < 70)] = "Bearing Fault"
health[(blade_torque > 32) & (motor_vib > 1.2)] = "Load Imbalance"
health[(np.abs(I_A - I_B) > 4)] = "Electrical Fault"

# ---- RUL ----
blade_rul = gen_piecewise_rul(blade_state, blade_rul_ranges)
motor_rul = gen_piecewise_rul(health, motor_rul_ranges)

# Power consumption
power_factor = 0.9 + 0.05*smooth_noise(N, 0.1)
power_consumption = (np.sqrt(3) * (I_A+I_B+I_C)/3 * 400 * power_factor / 1000).round(2)

# ---- DataFrames ----
motor_df = pd.DataFrame({
    "Timestamp": t,
    "PhaseA_Current": I_A.round(3),
    "PhaseB_Current": I_B.round(3),
    "PhaseC_Current": I_C.round(3),
    "Power_Consumption": power_consumption,
    "Power_Factor": power_factor.round(3),
    "Vibration": motor_vib.round(3),
    "Temp": motor_temp.round(3),
    "Speed": motor_speed.round(1),
    "Health_Status": health,
    "RUL": motor_rul.round(1)
})

blade_df = pd.DataFrame({
    "Timestamp": t,
    "Blade_Type": blade_type_series,
    "Vibration": blade_vib.round(3),
    "Torque": blade_torque.round(3),
    "Speed": blade_speed.round(1),
    "Noise": blade_noise.round(2),
    "Temp": blade_temp.round(2),
    "Condition": blade_state,
    "RUL": blade_rul.round(1)
})

combined_df = pd.DataFrame({
    "Timestamp": t,
    "Motor_Current": (0.333*(I_A+I_B+I_C)).round(3),
    "Motor_Vibration": motor_vib.round(3),
    "Motor_Temp": motor_temp.round(3),
    "Motor_Power": power_consumption,
    "Blade_Type": blade_type_series,
    "Blade_Vibration": blade_vib.round(3),
    "Blade_Torque": blade_torque.round(3),
    "Blade_Speed": blade_speed.round(1),
    "Blade_Noise": blade_noise.round(2),
    "Blade_Temp": blade_temp.round(2),
    "Health_Status": health,
    "Condition": blade_state,
    "RUL_Motor": motor_rul.round(1),
    "RUL_Blade": blade_rul.round(1)
})

# ---- Save ----
motor_path = "motor_dataset_consistent.csv"
blade_path = "blade_dataset_consistent.csv"
combined_path = "combined_dataset_consistent.csv"

motor_df.to_csv(motor_path, index=False)
blade_df.to_csv(blade_path, index=False)
combined_df.to_csv(combined_path, index=False)

print(motor_path, blade_path, combined_path)


motor_dataset_consistent.csv blade_dataset_consistent.csv combined_dataset_consistent.csv
